# Mapping main effects to anatomical regions

In this notebook we are going to get a map representing the main effect in the MCSE study, extracts the peak coordinates from it and then fetch anatomical labels for these regions.

We will first load the image we obtained from the previous group analysis. We are only interested in the low-high salience contrast.

In [ ]:
from pathlib import Path
from nilearn.image import load_img

filename = Path("results") / "group_low-high salience_z_score.nii.gz"
z_map = load_img(filename)

Now we can use the function `get_clusters_table` from the reporting module to get cluster information

In [ ]:
from nilearn.reporting import get_clusters_table

# we need to threshold the maps. We keep the threshold that controls  the FDR at 5% 
threshold = 3.06
table = get_clusters_table(
    z_map,
    stat_threshold=threshold,
    cluster_threshold=20
)
table

We see 37 main peaks organized in 14 clusters.
Now let us try to make sense of these clusters by observing them in the Julich atlas space.

To fetch the Julich atlas from EBRAINS we rely on the `siibra` library. We first need to import it.

In [ ]:
try:
    import siibra
except:
    !pip install -U siibra
    import siibra
print(siibra.__version__)
assert siibra.__version__ >= "0.4a33"

Now we can fetch the Julich-Brain parcellation as a labelled map in MNI space 

In [ ]:
julich_brain = siibra.parcellations.JULICH_BRAIN_PROBABILISTIC_CYTOARCHITECTONIC_MAPS_V2_9
print(julich_brain.description)

Now we can get the labelled map, which is a maximum probability map, and store it as an object. Since it is originally provided with split hemispheres we can use the `compress` method of the object to obtain a single relabelled map.

In [ ]:
# the original maximum probability map object with split hemispheres
julich_mpm = julich_brain.get_map(
    space=siibra.spaces.MNI_152_ICBM_2009C_NONLINEAR_ASYMMETRIC,
    maptype=siibra.MapType.LABELLED
)
# To work with it in a memory-friendly fashion, we resample the atlas to the fMRI resolution
from nilearn.image import resample_to_img

# single-volume compressed version
julich_mpm_compressed = julich_mpm.compress()

# load as a nifti image object 
parcellation_map_niimg = julich_mpm_compressed.fetch()

You can access the regions defined in the parcellation with the `regions` attribute

In [ ]:
julich_mpm_compressed.regions

In [ ]:
len(julich_mpm_compressed.regions)

We can see that there are **294** regions defined in the parcellation

With Nilearn we can visualize the thresholded statistical map on the Julich parcellation map

In [ ]:
from nilearn import plotting

plotting.view_img(z_map, bg_img=parcellation_map_niimg, threshold=threshold)

Now let's get the coordinate indexes from the peak coordinates of the clusters as defined in the cluster table and
use these indexes to get the labels from the parcellation that are associated with the peak coordinates.

In [ ]:
import numpy as np 
from nilearn.image import coord_transform

labels = []
for x, y, z in zip(table["X"], table["Y"], table["Z"]):
    # get index of coord using inverse affine
    coord_idx = coord_transform(x, y, z, np.linalg.inv(parcellation_map_niimg.affine))
    coord_idx = tuple(int(c) for c in coord_idx)
    labels.append(parcellation_map_niimg.get_fdata()[coord_idx])

Finally, we can map the labels back to the atlas to get the region names. Note that there are activations that fall outside the regions defined by the atlas. These will be returned as None in the mapping.

In [ ]:
region_names = []
for label_ in labels:
    region_names.append(julich_mpm_compressed.get_region(int(label_)))

For easy visualization of the relationship betwen the cluster peak coordinates and regions, we can add the region names to the cluster table as a column and print it

In [ ]:
table["Region"] = region_names
table